In [24]:
'''5月20日(土)は0の付く日です。
過去3回分の0の付く日のデータを取得し、分析しました。
結果は以下の通りです。
・0の付く日の平均回転数は、0.0回転です。
・0の付く日の平均差枚数は、0.0枚です。
'''

'5月20日(土)は0の付く日です。\n過去3回分の0の付く日のデータを取得し、分析しました。\n結果は以下の通りです。\n・0の付く日の平均回転数は、0.0回転です。\n・0の付く日の平均差枚数は、0.0枚です。\n'

In [33]:
str(target_day_list)

"['2023-05-16', '2023-05-06', '2023-04-26']"

In [23]:
from flask import Flask, render_template, request
from datetime import date, timedelta,datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

user_data = {}
user_data['tenpo-name'] = 'マルハン新宿東宝ビル店'
user_data['n-times'] = '3'
user_data['target-date'] = '2023-05-16'
print(user_data)
print(user_data['tenpo-name'],user_data['n-times'],user_data['target-date'])
target_day_list = []
number = 0
today = date.today()
target_number:int = str(user_data['target-date'][-1])
for i in range(int(user_data['n-times'])):
    while True:
        #print(str(date.today() - timedelta(days=number))[-1])
        if target_number == str(today - timedelta(days=number))[-1]:
            target_day = today - timedelta(days=number)
            print('取得日',target_day)
            target_day_str = target_day.strftime('%Y-%m-%d')
            target_day_list.append(target_day_str)
            number += 1
            break
        else: 
            pass
        number += 1
target_day_list.reverse()
concat_df_list = []
urls = []
for serch_date in target_day_list:
    search_url = url = f"https://ana-slo.com/{serch_date}-{user_data['tenpo-name']}-data/"
    urls.append(search_url)

with ThreadPoolExecutor(3) as executor:
    results = list(executor.map(requests.get, urls))
print(results)

for search_response,target_day in zip(results, target_day_list):
    soup = BeautifulSoup(search_response.text, "lxml")
    elem = soup.select('#all_data_block')
    dfs = pd.read_html(str(elem))
    for df in dfs:
        if '機種名' in list(df.columns):
            tmp_df = df
            tmp_df['日付'] = target_day
            #tmp_df['機種名'] = tmp_df['機種名'].map(removal_text)
            break
    concat_df_list.append(df)

concat_df = pd.concat(concat_df_list,axis=0)
for column_name in ['合成確率','BB確率','RB確率','台番号','ART確率']:
    try:
        concat_df = concat_df.drop([column_name],axis=1)
    except:
        pass

concat_df = concat_df.groupby(['日付','機種名']).mean().sort_values('差枚',ascending=False)
concat_df['差枚'] =concat_df['差枚'].astype(int)
concat_df['G数'] =concat_df['G数'].astype(int)
concat_df['BB'] =concat_df['BB'].astype(int)
concat_df['RB'] =concat_df['RB'].astype(int)
concat_df = concat_df.reset_index()
concat_df

{'tenpo-name': 'マルハン新宿東宝ビル店', 'n-times': '3', 'target-date': '2023-05-16'}
マルハン新宿東宝ビル店 3 2023-05-16
取得日 2023-05-16
取得日 2023-05-06
取得日 2023-04-26
[<Response [200]>, <Response [200]>, <Response [200]>]


,日付,機種名,G数,差枚,BB,RB
0,2023-05-16,ニューゲッターマウス,7341,3100,44,30
1,2023-04-26,とある科学の超電磁砲,2355,3080,118,6
2,2023-05-06,劇場版 魔法少女まどか☆マギカ[前編]始まりの物語/[後編]永遠の物語,5543,2866,136,0
3,2023-05-06,アクエリオン ALL STARS,4587,2300,58,15
4,2023-05-16,とある科学の超電磁砲,4692,2260,152,17
...,...,...,...,...,...,...
126,2023-04-26,北斗の拳 宿命,1353,-1150,5,2
127,2023-05-06,緋弾のアリアII,1594,-1233,13,2
128,2023-05-16,マッピー,2728,-1400,6,8
129,2023-04-26,ニューゲッターマウス,2942,-1500,7,7


In [30]:
results

[<Response [200]>, <Response [200]>, <Response [200]>]

In [29]:
target_day_list

['2023-04-26', '2023-05-06', '2023-05-16']

In [8]:
touhou_df = df[df['店舗名'] == 'マルハン新宿東宝ビル店']
touhou_df[['日付', 'Nの日', '都道府県', '店舗名', '機種名', '台番号', '末尾', 'G数', '差枚',
       'BB', 'RB', 'BB確率', 'RB確率', '合成確率']]

,日付,Nの日,都道府県,店舗名,機種名,台番号,末尾,G数,差枚,BB,RB,BB確率,RB確率,合成確率
93054,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,アイムジャグラーEX-TP,917,7,2992.0,-800,8.0,7.0,1/374.0,1/427.4,1/199.5
93055,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,アイムジャグラーEX-TP,918,8,6365.0,100,23.0,21.0,1/276.7,1/303.1,1/144.7
93056,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,アイムジャグラーEX-TP,919,9,8552.0,-1300,25.0,30.0,1/342.1,1/285.1,1/155.5
93057,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,アイムジャグラーEX-TP,920,0,5093.0,1800,26.0,13.0,1/195.9,1/391.8,1/130.6
93058,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,アイムジャグラーEX-TP,921,1,6930.0,1600,31.0,23.0,1/223.5,1/301.3,1/128.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93541,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,ゼーガペイン2,995,5,0.0,0,0.0,0.0,1/0.0,1/0.0,1/0.0
93542,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,閃乱カグラ BURST UP,996,6,0.0,0,0.0,0.0,1/0.0,1/0.0,1/0.0
93543,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,ハイスクールD×D2 ハーレム王に俺はなる,997,7,0.0,0,0.0,0.0,1/0.0,1/0.0,1/0.0
93544,2022/12/09 00:00:00,9,東京都,マルハン新宿東宝ビル店,ドリフターズ,998,8,0.0,0,0.0,0.0,1/0.0,1/0.0,1/0.0
